In [1]:
%load_ext rich

In [2]:
from corpus_x import setup_x # type: ignore
from sqlpyd import Connection
c = Connection(DatabasePath="x.db", WAL=True)
setup_x(c)

Validation needed detail=PosixPath('/Users/mv/code/corpus/statutes/ra/9154/details.yaml'); e=ValidationError(model='StatuteUnit', errors=[{'loc': ('content',), 'msg': 'ensure this value has at most 100000 characters', 'type': 'value_error.any_str.max_length', 'ctx': {'limit_value': 100000}}])
Validation needed detail=PosixPath('/Users/mv/code/corpus/statutes/ra/9486/details.yaml'); e=ValidationError(model='StatuteUnit', errors=[{'loc': ('content',), 'msg': 'ensure this value has at most 100000 characters', 'type': 'value_error.any_str.max_length', 'ctx': {'limit_value': 100000}}])
Generic exception on insertion: e=SyntaxWarning("idx='1918' invalid serial of StatuteSerialCategory.RulesOfCourt")


In [ ]:
from sqlpyd import Connection
c = Connection(DatabasePath="x.db", WAL=True)

In [ ]:
from corpus_pax.utils import delete_tables_with_prefix
delete_tables_with_prefix(c=c, target_prefixes=["lex_tbl"])

In [ ]:
Statute.make_tables(c)

In [ ]:
Statute.add_rows(c)

In [ ]:
from corpus_x.statutes import StatuteFoundInUnit
StatuteFoundInUnit.update_statute_ids(c)  

In [ ]:
from corpus_x.codifications import Codification, CodeRow
Codification.make_tables(c)

In [ ]:
Codification.add_rows(c)

## Corpus-X

### Preparatory steps from files to db

The pre-processed data can now be used to insert related Statutes and Citations of each Opinion back into the database. 

The statute and inclusion tables need to be created before the pre-processed data can be inserted.

In [ ]:
from corpus_x.inclusions import Inclusion
Inclusion.make_tables(c) # note that statutes need to exist first

### Move content from files to db

Collect the pre-processed data and insert the same into the created database tables. 

Estimate at the end of 2022 (factors to consider):

1. the last time data was scraped as raw files,
2. the time separate opinions were manually included

Result: about ~484k `CitationsInOpinions` and ~99k `StatutesInOpinions` records. 

In [ ]:
from corpus_x.inclusions import populate_db_with_inclusions
populate_db_with_inclusions(c) 

### Ensure existence of component elements

What exists in the database are records of statutes but not the statutes themselves. In other words, the foreign key included in the `StatuteInOpinions` table does not yet have a counterpart in the `StatuteRow` table. 

Note that the `CitationInOpinions` will have a counterpart in the `DecisionRow` table since this was processed first.

In [ ]:
from corpus_x.inclusions import StatuteInOpinion, CitationInOpinion
StatuteInOpinion.add_statutes(c) # takes 2-3 minutes to store 500 objects
StatuteInOpinion.update_statute_ids(c)
CitationInOpinion.update_decision_ids(c)

In [ ]:
from corpus_x.inclusions import StatuteInOpinion, CitationInOpinion

In [ ]:
StatuteInOpinion.add_statutes(c)

In [ ]:
StatuteInOpinion.update_statute_ids(c)

In [ ]:
CitationInOpinion.update_decision_ids(c)

With `StatuteRow` and `CitationRow` tables already in existence, can proceed to add the `CodeRow` table.

### Add Codifications 

In [ ]:
from corpus_x.codifications import Codification, CodeStatuteEvent
Codification.make_tables(c) 
Codification.add_rows(c) # takes about 1-2 minutes


Determine Codifications that are missing affector paths, i.e. improper use of `item`, `caption`, `content` in matching an event to a Statute unit.

In [ ]:
from corpus_x.codifications import CodeStatuteEvent
if matches := CodeStatuteEvent.fetch_unmaterialized(c):
    print(f"Violating {len(matches)=}; review violators via SQL.")